In [1]:
import sys
sys.path.append('..')
from MPRA_predict.utils import *

In [3]:
model_track_info_df = pd.read_csv('../data/Borzoi/targets_human.txt', sep='\t', index_col=0)
model_track_info_df = model_track_info_df.reset_index()
model_track_info_df

,index,identifier,file,clip,clip_soft,scale,sum_stat,strand_pair,description
0,0,CNhs10608+,/home/drk/tillage/datasets/human/cage/fantom/C...,768,384,1.000,sum,1,CAGE:Clontech Human Universal Reference Total ...
1,1,CNhs10608-,/home/drk/tillage/datasets/human/cage/fantom/C...,768,384,1.000,sum,0,CAGE:Clontech Human Universal Reference Total ...
2,2,CNhs10610+,/home/drk/tillage/datasets/human/cage/fantom/C...,768,384,1.000,sum,3,CAGE:SABiosciences XpressRef Human Universal T...
3,3,CNhs10610-,/home/drk/tillage/datasets/human/cage/fantom/C...,768,384,1.000,sum,2,CAGE:SABiosciences XpressRef Human Universal T...
4,4,CNhs10612+,/home/drk/tillage/datasets/human/cage/fantom/C...,768,384,1.000,sum,5,CAGE:Universal RNA - Human Normal Tissues Bioc...
...,...,...,...,...,...,...,...,...,...
7606,7606,GTEX-13FTX-1026-SM-5J2O5.1,/home/drk/tillage/datasets/human/rna/recount3/...,768,384,0.010,sum_sqrt,7606,RNA:uterus
7607,7607,GTEX-1MA7W-1526-SM-DHXKS.1,/home/drk/tillage/datasets/human/rna/recount3/...,768,384,0.010,sum_sqrt,7607,RNA:uterus
7608,7608,GTEX-11EMC-1926-SM-5A5JU.1,/home/drk/tillage/datasets/human/rna/recount3/...,768,384,0.010,sum_sqrt,7608,RNA:vagina
7609,7609,GTEX-12WSB-2426-SM-5EGJC.1,/home/drk/tillage/datasets/human/rna/recount3/...,768,384,0.010,sum_sqrt,7609,RNA:vagina


In [4]:
def parse_description(desc):
    parts = desc.split(':')
    if len(parts) == 2:
        assay = parts[0]
        cell_name = parts[1]
    elif len(parts) == 3:
        assay = ':'.join(parts[:2])
        cell_name = parts[2]
    else:
        return desc, None
    return assay, cell_name

def rename_assay(x):
    if x == 'DNASE':
        return 'DNase'
    elif x.startswith('CHIP:'):
        return x.replace('CHIP:', '')
    else:
        return x

model_track_info_df[['assay', 'cell_name']] = model_track_info_df['description'].apply(parse_description).apply(pd.Series)
model_track_info_df['assay'] = model_track_info_df['assay'].apply(rename_assay)

In [5]:
cell_types = ['K562', 'HepG2', 'SK-N-SH', 'HCT116', 'A549']
assays = ['DNase', 'H3K4me3', 'H3K27ac', 'CTCF', 'RNA']

for cell_type in cell_types:
    mask = model_track_info_df['cell_name'].str.contains(cell_type, na=False)
    filtered_df = model_track_info_df[mask]
    print(f"=== {cell_type} ===")
    print(filtered_df['cell_name'].value_counts())

=== K562 ===
cell_name
K562                                                                              365
K562 eGFP- genetically modified using stable transfection                          52
K562 eGFP- genetically modified using CRISPR                                       15
K562 treated with interferon alpha for 30 minutes                                   7
K562 treated with interferon alpha for 6 hours                                      6
K562 cytosolic fraction                                                             6
K562 nuclear fraction                                                               6
K562 treated with interferon gamma for 6 hours                                      5
K562 treated with interferon gamma for 30 minutes                                   4
K562 membrane fraction                                                              4
K562 insoluble cytoplasmic fraction                                                 4
K562 nucleolus fraction        

In [6]:
df_pivot = model_track_info_df.pivot_table(
    values="index", 
    index="cell_name", 
    columns="assay", 
    aggfunc=list,
)
df_pivot = df_pivot.map(lambda x: x if isinstance(x, list) else [])
df_pivot.map(len)

assay,ADNP,AEBP2,AFF1,AHR,ARHGAP35,ARID1B,ARID2,ARID3A,ARID4B,ARNT,ASCL2,ASH1L,ASH2L,ATAC,ATF1,ATF2,ATF3,ATF4,ATF5,ATF7,ATM,ATOH1,BACH1,BARHL1,BARX2,BATF,BCL11A,BCL11B,BCL3,BCL6,BCL6B,BCLAF1,BCOR,BHLHE40,BMAL1,BMI1,BRCA1,BRD4,BRD9,C11orf30,CAGE,CAGE:B lymphoblastoid cell line,CAGE:Burkitt's lymphoma cell line,CAGE:Ewing's sarcoma cell line,CAGE:Hodgkin's lymphoma cell line,CAGE:Krukenberg tumor cell line,CAGE:NK T cell leukemia cell line,CAGE:Wilms' tumor cell line,CAGE:acantholytic squamous carcinoma cell line,CAGE:acute lymphoblastic leukemia (B-ALL) cell line,CAGE:acute lymphoblastic leukemia (T-ALL) cell line,CAGE:acute myeloid leukemia (FAB M0) cell line,CAGE:acute myeloid leukemia (FAB M1) cell line,CAGE:acute myeloid leukemia (FAB M2) cell line,CAGE:acute myeloid leukemia (FAB M3) cell line,CAGE:acute myeloid leukemia (FAB M4) cell line,CAGE:acute myeloid leukemia (FAB M4eo) cell line,CAGE:acute myeloid leukemia (FAB M5) cell line,CAGE:acute myeloid leukemia (FAB M6) cell line,CAGE:acute myeloid leukemia (FAB M7) cell line,CAGE:adenocarcinoma cell line,CAGE:adrenal cortex adenocarcinoma cell line,CAGE:adult T-cell leukemia cell line,CAGE:alveolar cell carcinoma cell line,CAGE:anaplastic carcinoma cell line,CAGE:anaplastic large cell lymphoma cell line,CAGE:anaplastic squamous cell carcinoma cell line,CAGE:argyrophil small cell carcinoma cell line,CAGE:astrocytoma cell line,CAGE:b cell line,CAGE:basal cell carcinoma cell line,CAGE:bile duct carcinoma cell line,CAGE:biphenotypic B myelomonocytic leukemia cell line,CAGE:bone marrow stromal cell line,CAGE:breast carcinoma cell line,CAGE:bronchial squamous cell carcinoma cell line,CAGE:bronchioalveolar carcinoma cell line,CAGE:bronchogenic carcinoma cell line,CAGE:carcinoid cell line,CAGE:carcinosarcoma cell line,CAGE:cervical cancer cell line,CAGE:cholangiocellular carcinoma cell line,CAGE:chondrosarcoma cell line,CAGE:choriocarcinoma cell line,CAGE:chronic lymphocytic leukemia (T-CLL) cell line,CAGE:chronic myeloblastic leukemia (CML) cell line,CAGE:chronic myelogenous leukemia (CML) cell line,CAGE:chronic myelogenous leukemia cell line,CAGE:clear cell carcinoma cell line,CAGE:colon carcinoma cell line,CAGE:cord blood derived cell line,CAGE:diffuse large B-cell lymphoma cell line,CAGE:ductal cell carcinoma cell line,CAGE:embryonic kidney cell line,CAGE:embryonic pancreas cell line,CAGE:endometrial carcinoma cell line,CAGE:endometrial stromal sarcoma cell line,CAGE:endometrioid adenocarcinoma cell line,CAGE:epidermoid carcinoma cell line,CAGE:epithelioid sarcoma cell line,CAGE:epitheloid carcinoma cell line,CAGE:extraskeletal myxoid chondrosarcoma cell line,CAGE:fibrosarcoma cell line,CAGE:fibrous histiocytoma cell line,CAGE:gall bladder carcinoma cell line,CAGE:gastric adenocarcinoma cell line,CAGE:gastric cancer cell line,CAGE:gastrointestinal carcinoma cell line,CAGE:giant cell carcinoma cell line,CAGE:glassy cell carcinoma cell line,CAGE:glioblastoma cell line,CAGE:glioma cell line,CAGE:granulosa cell tumor cell line,CAGE:hairy cell leukemia cell line,CAGE:hepatic mesenchymal tumor cell line,CAGE:hepatoblastoma cell line,CAGE:hepatocellular carcinoma cell line,CAGE:hepatoma cell line,CAGE:hereditary spherocytic anemia cell line,CAGE:keratoacanthoma cell line,CAGE:large cell lung carcinoma cell line,CAGE:large cell non-keratinizing squamous carcinoma cell line,CAGE:leiomyoblastoma cell line,CAGE:leiomyoma cell line,CAGE:leiomyosarcoma cell line,CAGE:lens epithelial cell line,"CAGE:leukemia, chronic megakaryoblastic cell line",CAGE:liposarcoma cell line,CAGE:lung adenocarcinoma cell line,"CAGE:lung adenocarcinoma, papillary cell line",CAGE:lymphangiectasia cell line,"CAGE:lymphoma, malignant, hairy B-cell cell line",CAGE:malignant trichilemmal cyst cell line,CAGE:maxillary sinus tumor cell line,CAGE:medulloblastoma cell line,CAGE:melanoma cell line,CAGE:meningioma cell line,CAGE:merkel cell carcinoma cell line,CAGE:mesenchymal stem cell line,CAGE:mesodermal tumor cell lin

In [7]:
cell_types = ['K562', 'HepG2', 'SK-N-SH', 'HCT116', 'A549', 'SK-N-SH treated with 6 uM all-trans-retinoic acid for 48 hours']
assays = ['DNase', 'H3K4me3', 'H3K27ac', 'CTCF', 'RNA']
df_pivot.loc[cell_types, assays].map(len)

assay,DNase,H3K4me3,H3K27ac,CTCF,RNA
cell_name,,,,,
K562,4,4,0,4,23
HepG2,3,3,0,1,13
SK-N-SH,0,2,2,2,3
HCT116,1,2,2,3,3
A549,1,1,0,2,2
SK-N-SH treated with 6 uM all-trans-retinoic acid for 48 hours,1,1,0,1,2


In [8]:
assays = ['DNase', 'H3K4me3', 'H3K27ac', 'CTCF', 'CAGE']
cell_types = ['K562', 'HepG2', 'SK-N-SH', 'HCT116', 'A549']
df_track = df_pivot.loc[cell_types, assays]
df_track.loc['SK-N-SH', 'DNase'] = df_track.loc['SK-N-SH', 'DNase'] + df_pivot.loc['SK-N-SH treated with 6 uM all-trans-retinoic acid for 48 hours', 'DNase']
df_track

assay,DNase,H3K4me3,H3K27ac,CTCF,CAGE
cell_name,,,,,
K562,"[1397, 1398, 1399, 1901]","[2199, 2708, 2881, 4930]",[],"[2195, 2510, 2709, 2841]",[]
HepG2,"[1303, 1367, 1510]","[2215, 2680, 4663]",[],[2402],[]
SK-N-SH,[1422],"[2935, 5833]","[2937, 4633]","[2865, 4550]",[]
HCT116,[1364],"[2671, 3920]","[2873, 4908]","[2553, 2670, 3666]",[]
A549,[1323],[2600],[],"[2582, 2740]",[]


In [9]:
# pred_array = load_h5('outputs/Gosai_MPRA_Borzoi_pred.h5')
pred_array = load_h5('outputs/Gosai_MPRA_Borzoi_pred_1.h5')
print(pred_array.shape)

file outputs/Gosai_MPRA_Borzoi_pred_1.h5 has keys: ['data']
(760679, 8, 7611)


In [10]:
MPRA_df = pd.read_csv('../data/Gosai_MPRA/Gosai_MPRA_my_processed_data_len200_norm.csv')
print(MPRA_df.shape)

(760679, 14)


In [11]:
VEF_df = pd.DataFrame()
for i, cell_type in enumerate(cell_types):
    for j, assay in enumerate(assays):
        indice = df_track.loc[cell_type, assay]
        if isinstance(indice, list) and len(indice) > 0:
            pred = pred_array[:, 1:7, indice].mean(2).mean(1)
            VEF_df[f'{cell_type}_{assay}'] = pred
        else:
            VEF_df[f'{cell_type}_{assay}'] = np.nan

In [12]:
for j, assay in enumerate(assays):
    pearson_df = pd.DataFrame()
    for ct1 in cell_types:
        for ct2 in cell_types:
            true = MPRA_df[f'{ct1}']
            pred = VEF_df[f'{ct2}_{assay}']
            r, _ = pearson(pred, true)
            pearson_df.loc[f'{ct2}_{assay}', f'{ct1}'] = r
    print(assay)
    print(pearson_df)

DNase
               K562  HepG2  SK-N-SH  HCT116  A549
K562_DNase    0.439  0.405    0.410   0.427 0.329
HepG2_DNase   0.392  0.440    0.393   0.427 0.315
SK-N-SH_DNase 0.321  0.314    0.331   0.342 0.260
HCT116_DNase  0.335  0.325    0.328   0.358 0.325
A549_DNase    0.357  0.368    0.350   0.374 0.346
H3K4me3
                 K562  HepG2  SK-N-SH  HCT116  A549
K562_H3K4me3    0.348  0.344    0.363   0.360 0.250
HepG2_H3K4me3   0.226  0.266    0.250   0.250 0.182
SK-N-SH_H3K4me3 0.286  0.281    0.300   0.297 0.193
HCT116_H3K4me3  0.339  0.326    0.352   0.369 0.275
A549_H3K4me3    0.293  0.294    0.302   0.307 0.268
after remove nan, len(x) < 2, pearson = nan
after remove nan, len(x) < 2, pearson = nan
after remove nan, len(x) < 2, pearson = nan
after remove nan, len(x) < 2, pearson = nan
after remove nan, len(x) < 2, pearson = nan
after remove nan, len(x) < 2, pearson = nan
after remove nan, len(x) < 2, pearson = nan
after remove nan, len(x) < 2, pearson = nan
after remove nan, len(

In [13]:
pearson_df = pd.DataFrame(dtype=float)
for i, cell_type in enumerate(cell_types):
    for j, assay in enumerate(assays):
        pred = VEF_df[f'{cell_type}_{assay}']
        true = MPRA_df[f'{cell_type}']
        r, _ = pearson(pred, true)
        pearson_df.loc[cell_type, assay] = r
print(pearson_df)

after remove nan, len(x) < 2, pearson = nan
after remove nan, len(x) < 2, pearson = nan
after remove nan, len(x) < 2, pearson = nan
after remove nan, len(x) < 2, pearson = nan
after remove nan, len(x) < 2, pearson = nan
after remove nan, len(x) < 2, pearson = nan
after remove nan, len(x) < 2, pearson = nan
after remove nan, len(x) < 2, pearson = nan
         DNase  H3K4me3  H3K27ac  CTCF  CAGE
K562     0.439    0.348      NaN 0.212   NaN
HepG2    0.440    0.266      NaN 0.171   NaN
SK-N-SH  0.331    0.300    0.178 0.118   NaN
HCT116   0.358    0.369    0.349 0.153   NaN
A549     0.346    0.268      NaN 0.166   NaN


In [14]:
VEF_df = VEF_df.astype('float32')
VEF_df.describe()

,K562_DNase,K562_H3K4me3,K562_H3K27ac,K562_CTCF,K562_CAGE,HepG2_DNase,HepG2_H3K4me3,HepG2_H3K27ac,HepG2_CTCF,HepG2_CAGE,SK-N-SH_DNase,SK-N-SH_H3K4me3,SK-N-SH_H3K27ac,SK-N-SH_CTCF,SK-N-SH_CAGE,HCT116_DNase,HCT116_H3K4me3,HCT116_H3K27ac,HCT116_CTCF,HCT116_CAGE,A549_DNase,A549_H3K4me3,A549_H3K27ac,A549_CTCF,A549_CAGE
count,760679.000,760679.000,0.000,760679.000,0.000,760679.000,760679.000,0.000,760679.000,0.000,760679.000,760679.000,760679.000,760679.000,0.000,760679.000,760679.000,760679.000,760679.000,0.000,760679.000,760679.000,0.000,760679.000,0.000
mean,0.052,0.619,NaN,0.985,NaN,0.061,0.522,NaN,1.026,NaN,0.064,0.847,0.729,1.004,NaN,0.049,0.686,0.821,1.024,NaN,0.049,0.311,NaN,0.996,NaN
std,0.105,0.495,NaN,1.479,NaN,0.064,0.343,NaN,0.971,NaN,0.139,0.338,0.708,1.186,NaN,0.075,0.611,0.854,1.144,NaN,0.075,0.474,NaN,0.919,NaN
min,0.001,0.004,NaN,0.003,NaN,0.003,0.006,NaN,0.011,NaN,0.003,0.036,0.008,0.023,NaN,0.003,0.010,0.014,0.013,NaN,0.003,0.001,NaN,0.014,NaN
25%,0.022,0.412,NaN,0.585,NaN,0.031,0.369,NaN,0.649,NaN,0.034,0.706,0.392,0.754,NaN,0.033,0.478,0.557,0.716,NaN,0.027,0.194,NaN,0.653,NaN
50%,0.033,0.617,NaN,0.887,NaN,0.045,0.553,NaN,1.052,NaN,0.045,0.875,0.578,1.012,NaN,0.043,0.706,0.741,1.059,NaN,0.039,0.309,NaN,0.876,NaN
75%,0.055,0.744,NaN,1.137,NaN,0.073,0.635,NaN,1.271,NaN,0.059,0.980,0.816,1.107,NaN,0.051,0.812,0.888,1.191,NaN,0.056,0.357,NaN,1.309,NaN
max,5.559,33.812,NaN,47.062,NaN,5.484,42.125,NaN,30.438,NaN,13.414,31.219,24.641,42.812,NaN,4.719,44.125,32.938,40.312,NaN,5.082,55.406,NaN,33.625,NaN


In [15]:
VEF_df.to_csv('../data/Gosai_MPRA/Gosai_MPRA_Borzoi_VEF_raw.tsv', sep='\t', index=False)

In [3]:
MPRA_df = pd.read_csv('../data/Gosai_MPRA/Gosai_MPRA_my_processed_data_len200_norm.csv')
VEF_df = pd.read_csv('../data/Gosai_MPRA/Gosai_MPRA_Borzoi_VEF_raw.tsv', sep='\t')
VEF_df.describe()

,K562_DNase,K562_H3K4me3,K562_H3K27ac,K562_CTCF,K562_CAGE,HepG2_DNase,HepG2_H3K4me3,HepG2_H3K27ac,HepG2_CTCF,HepG2_CAGE,SK-N-SH_DNase,SK-N-SH_H3K4me3,SK-N-SH_H3K27ac,SK-N-SH_CTCF,SK-N-SH_CAGE,HCT116_DNase,HCT116_H3K4me3,HCT116_H3K27ac,HCT116_CTCF,HCT116_CAGE,A549_DNase,A549_H3K4me3,A549_H3K27ac,A549_CTCF,A549_CAGE
count,760679.000,760679.000,0.000,760679.000,0.000,760679.000,760679.000,0.000,760679.000,0.000,760679.000,760679.000,760679.000,760679.000,0.000,760679.000,760679.000,760679.000,760679.000,0.000,760679.000,760679.000,0.000,760679.000,0.000
mean,0.052,0.619,NaN,0.985,NaN,0.061,0.522,NaN,1.026,NaN,0.064,0.847,0.729,1.004,NaN,0.049,0.686,0.821,1.024,NaN,0.049,0.311,NaN,0.996,NaN
std,0.105,0.496,NaN,1.480,NaN,0.064,0.343,NaN,0.971,NaN,0.139,0.338,0.708,1.187,NaN,0.075,0.611,0.855,1.145,NaN,0.075,0.475,NaN,0.919,NaN
min,0.001,0.004,NaN,0.003,NaN,0.003,0.006,NaN,0.011,NaN,0.003,0.036,0.008,0.023,NaN,0.003,0.010,0.014,0.013,NaN,0.003,0.001,NaN,0.014,NaN
25%,0.022,0.412,NaN,0.585,NaN,0.031,0.369,NaN,0.649,NaN,0.034,0.706,0.392,0.754,NaN,0.033,0.478,0.557,0.716,NaN,0.027,0.194,NaN,0.653,NaN
50%,0.033,0.617,NaN,0.887,NaN,0.045,0.553,NaN,1.052,NaN,0.045,0.875,0.578,1.012,NaN,0.043,0.706,0.741,1.059,NaN,0.039,0.309,NaN,0.876,NaN
75%,0.055,0.744,NaN,1.137,NaN,0.073,0.635,NaN,1.271,NaN,0.059,0.980,0.816,1.107,NaN,0.051,0.812,0.888,1.191,NaN,0.056,0.357,NaN,1.309,NaN
max,5.559,33.812,NaN,47.062,NaN,5.484,42.125,NaN,30.438,NaN,13.414,31.219,24.641,42.812,NaN,4.719,44.125,32.938,40.312,NaN,5.082,55.406,NaN,33.625,NaN


In [4]:
VEF_df = np.log1p(VEF_df)
VEF_df.to_csv('../data/Gosai_MPRA/Gosai_MPRA_Borzoi_VEF_log1p.tsv', sep='\t', index=False)